<a href="https://colab.research.google.com/github/prasadv26/mind-map-generator-crewai/blob/main/mind_map_creator_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai litellm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.6 MB/s eta 0

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "***"

In [3]:
import json
from crewai import Agent, Task, Crew, Process
from litellm import completion

#AGENTS DEFINITION

In [5]:
concept_extractor = Agent(
    role="Concept Extractor",
    goal="Extract the key concepts and subtopics from a given topic or document.",
    backstory="A detail-oriented researcher skilled at identifying central and supporting ideas from long texts.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

hierarchy_builder = Agent(
    role="Hierarchy Builder",
    goal="Arrange extracted concepts into a clear mind map hierarchy with parent-child relationships.",
    backstory="An expert in organizing information and visualizing complex relationships.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

json_formatter = Agent(
    role="JSON Formatter",
    goal="Transform the hierarchical mind map into a clean JSON structure usable by visualization tools.",
    backstory="A developer specialized in data formatting and tool compatibility.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

image_prompt_generator = Agent(
    role="Image Prompt Generator",
    goal="Write a creative prompt to visualize the mind map as an image using AI art.",
    backstory="An expert in crafting detailed prompts for mind map visualization.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

TASKS

In [9]:
concept_extract_task = Task(
    description="Given the input topic or article: {input_text}, extract all major concepts and subtopics as clearly as possible. Output as a structured bullet list.",
    agent=concept_extractor,
    expected_output="- Concept 1\n  - Subconcept 1.1\n- Concept 2\n  - Subconcept 2.1\n  - Subconcept 2.2"
)

hierarchy_build_task = Task(
    description="Using the extracted concepts, organize them into a tree-like mind map structure, nesting subtopics under their main topics using indentation.",
    agent=hierarchy_builder,
    context=[concept_extract_task],
    expected_output="Root\n  Concept 1\n    Subconcept 1.1\n  Concept 2\n    Subconcept 2.1"
)

json_format_task = Task(
    description=(
        "Convert the mind map hierarchy into clean, properly-structured JSON suitable for visualization. "
        "Use this format: {'name': 'Root', 'children': [{'name': 'Concept 1', 'children': [...]}, ...]}."
    ),
    agent=json_formatter,
    context=[hierarchy_build_task],
    expected_output='{"name": "Root", "children": [...]}'
)

image_prompt_task = Task(
    description="Write a descriptive prompt to visualize the mind map structure as an image (DO NOT attempt to generate or render an image).",
    agent=image_prompt_generator,
    context=[hierarchy_build_task],
    expected_output="A detailed prompt describing the topic and structure, suitable for a mind map image generator."
)

crew

In [10]:
crew = Crew(
    agents=[concept_extractor, hierarchy_builder, json_formatter, image_prompt_generator],
    tasks=[concept_extract_task, hierarchy_build_task, json_format_task, image_prompt_task],
    process=Process.sequential,
    verbose=True
)

In [11]:
topic_or_article = "The impact of renewable energy on modern power grids"

result = crew.kickoff(inputs={"input_text": topic_or_article})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5d12eeb3-7cd7-4fab-996b-5054389f9bc0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Concept Extractor                                                                                       │
│                                                                                                                 │
│  Task: Given the input topic or article: The impact of renewable energy on modern power grids, extract all      │
│  major concepts and subtopics as clearly as possible. Output as a structured bullet list.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Concept Extractor                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - Impact of Renewable Energy on Power Grids                                                                    │
│    - Increased Efficiency                                                                                       │
│    - Reduction of Carbon Footprint                                                                              │
│  - Types of Renewable Energy                                                                                    │
│    - Solar Energy                                                                                               │
│      - Photovoltaics                                                                                            │
│      - Solar Thermal                                                                                            │
│    - Wind Energy                                                                                                │
│      - Onshore Wind Farms                                                                                       │
│      - Offshore Wind Farms                                                                                      │
│    - Hydropower                                                                                                 │
│      - Dam-Based Systems                                                                                        │
│      - Run-of-River Systems                                                                                     │
│    - Biomass Energy                                                                                             │
│  - Challenges to Integration                                                                                    │
│    - Grid Stability                                                                                             │
│    - Energy Storage Solutions                                                                                   │
│      - Batteries                                                                                                │
│      - Pumped Hydroelectric Storage                                                                             │
│    - Transmission Infrastructure                                                                                │
│  - Economic Impacts                                                                                             │
│    - Job Creation                                                                                               │
│    - Investment in Technology                                                                                   │
│  - Policy and Regulation                                                                                        │
│    - Government Incentives                                                                                      │
│    - International Agreements                                                                                   │
│  - Future Trends                                                                                                │
│    - Smart Grids                                                                                                │
│    - Microgrids                                                                                                 │
│    - Decentralized Energy Production                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 213a6d6c-286d-4c9c-b703-54cdae75caf0                                                                     │
│  Agent: Concept Extractor                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image Prompt Generator                                                                                  │
│                                                                                                                 │
│  Task: Write a descriptive prompt to visualize the mind map structure as an image (DO NOT attempt to generate   │
│  or render an image).                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image Prompt Generator                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Create a mind map visualization focused on the "Impact of Renewable Energy on Power Grids". At the center,     │
│  place the title "Impact of Renewable Energy on Power Grids" in bold, vibrant lettering, surrounded by a        │
│  circular node. From this central node, extend multiple branches leading to key themes:                         │
│                                                                                                                 │
│  1. **Increased Efficiency**: Use graphics of turbines and solar panels to symbolize enhanced energy            │
│  performance.                                                                                                   │
│  2. **Reduction of Carbon Footprint**: Include environmental icons, such as a globe with greenery, to           │
│  represent eco-friendliness.                                                                                    │
│                                                                                                                 │
│  Next, branch off to **Types of Renewable Energy**, subdividing into four major categories:                     │
│  - **Solar Energy**: This sub-branch contains two further divisions:                                            │
│    - **Photovoltaics**: Illustrate with solar cells.                                                            │
│    - **Solar Thermal**: Represent with thermal collectors.                                                      │
│                                                                                                                 │
│  - **Wind Energy**: Split into:                                                                                 │
│    - **Onshore Wind Farms**: Show images of land-based turbines.                                                │
│    - **Offshore Wind Farms**: Use ocean backgrounds with turbines.                                              │
│                                                                                                                 │
│  - **Hydropower**: This section divides into:                                                                   │
│    - **Dam-Based Systems**: Illustrate with large dams and flowing water.                                       │
│    - **Run-of-River Systems**: Depict rivers with power generation elements.                                    │
│                                                                                                                 │
│  - **Biomass Energy**: Represent with plant materials being converted into energy.                              │
│                                                                                                                 │
│  Then move to **Challenges to Integration**, featuring sub-branches that focus on:                              │
│  - **Grid Stability**: Visualize with interconnected networks.                                                  │
│  - **Energy Storage Solutions**: This includes further divisions:                                               │
│    - **Batteries**: Show modern battery designs.                                                                │
│    - **Pumped Hydroelectric Storage**: Depict water reservoirs and turbines.                                    │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 04497386-6d0d-4aa4-8ec9-4602ac12c6f4                                                                     │
│  Agent: Image Prompt Generator                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5d12eeb3-7cd7-4fab-996b-5054389f9bc0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Create a mind map visualization focused on the "Impact of Renewable Energy on Power Grids". At   │
│  the center, place the title "Impact of Renewable Energy on Power Grids" in bold, vibrant lettering,            │
│  surrounded by a circular node. From this central node, extend multiple branches leading to key themes:         │
│                                                                                                                 │
│  1. **Increased Efficiency**: Use graphics of turbines and solar panels to symbolize enhanced energy            │
│  performance.                                                                                                   │
│  2. **Reduction of Carbon Footprint**: Include environmental icons, such as a globe with greenery, to           │
│  represent eco-friendliness.                                                                                    │
│                                                                                                                 │
│  Next, branch off to **Types of Renewable Energy**, subdividing into four major categories:                     │
│  - **Solar Energy**: This sub-branch contains two further divisions:                                            │
│    - **Photovoltaics**: Illustrate with solar cells.                                                            │
│    - **Solar Thermal**: Represent with thermal collectors.                                                      │
│                                                                                                                 │
│  - **Wind Energy**: Split into:                                                                                 │
│    - **Onshore Wind Farms**: Show images of land-based turbines.                                                │
│    - **Offshore Wind Farms**: Use ocean backgrounds with turbines.                                              │
│                                                                                                                 │
│  - **Hydropower**: This section divides into:                                                                   │
│    - **Dam-Based Systems**: Illustrate with large dams and flowing water.                                       │
│    - **Run-of-River Systems**: Depict rivers with power generation elements.                                    │
│                                                                                                                 │
│  - **Biomass Energy**: Represent with plant materials being converted into energy.                              │
│                                                                                                                 │
│  Then move to **Challenges to Integration**, featuring sub-branches that focus on:                              │
│  - **Grid Stability**: Visualize with interconnected networks.                                                  │
│  - **Energy Storage Solutions**: This includes further divisions:                                               │
│    - **Batteries**: Show modern battery designs.                                                                │
│    - **Pumped Hydroelectric Storage**: Depict water re